*Credits: materials from this notebook belong to YSDA [Practical DL](https://github.com/yandexdataschool/Practical_DL) course. Special thanks for making them available online.*

# Lab assignment №1, part 1

This lab assignment consists of several parts. You are supposed to make some transformations, train some models, estimate the quality of the models and explain your results.

Several comments:
* Don't hesitate to ask questions, it's a good practice.
* No private/public sharing, please. The copied assignments will be graded with 0 points.
* Blocks of this lab will be graded separately.

## 1. Matrix differentiation

Since it easy to google every task please please please try to undestand what's going on. The "just answer" thing will be not counted, make sure to present derivation of your solution. It is absolutely OK if you found an answer on web then just exercise in $\LaTeX$ copying it into here.

Useful links:
[1](http://www.machinelearning.ru/wiki/images/2/2a/Matrix-Gauss.pdf)
[2](http://www.atmos.washington.edu/~dennis/MatrixCalculus.pdf)

## ex. 1

$$  
y = x^Tx,  \quad x \in \mathbb{R}^N
$$

$$
\frac{dy}{dx} = ?
$$

**Answer:**

$$
\frac{dy}{dx} = 2x
$$

In [ ]:
import numpy as np

def grad_y(x):
    return 2 * x

x = np.array([1.0, 2.0, 3.0])
grad_y(x)


array([2., 4., 6.])

## ex. 2

$$ y = tr(AB) \quad A,B \in \mathbb{R}^{N \times N} $$

$$
\frac{dy}{dA} = ?
$$

**Answer:**

Using
$$
y = \mathrm{tr}(AB) = \mathrm{tr}(BA),
$$
we obtain
$$
\frac{dy}{dA} = B^T.
$$

## ex. 3

$$  
y = x^TAc , \quad A\in \mathbb{R}^{N \times N}, x\in \mathbb{R}^{N}, c\in \mathbb{R}^{N}
$$

$$
\frac{dy}{dx} \quad \text{and} \quad \frac{dy}{dA}.
$$

**Answer:**

We can write
$$
y = x^T A c = \sum_{i=1}^N \sum_{j=1}^N x_i A_{ij} c_j.
$$

Then
$$
\frac{\partial y}{\partial x_i} = \sum_{j} A_{ij} c_j,
$$
so in vector form
$$
\frac{dy}{dx} = A c.
$$

For $A$:
$$
\frac{\partial y}{\partial A_{ij}} = x_i c_j,
$$
which gives
$$
\frac{dy}{dA} = x c^T.
$$

Hint for the latter (one of the ways): use *ex. 2* result and the fact
$$
tr(ABC) = tr (CAB)
$$

## ex. 4

Classic matrix factorization example. Given matrix $X$ you need to find $A$, $S$ to approximate $X$. This can be done by simple gradient descent iteratively alternating $A$ and $S$ updates.
$$
J = || X - AS ||_F^2  , \quad A\in \mathbb{R}^{N \times R} , \quad S\in \mathbb{R}^{R \times M}
$$
$$
\frac{dJ}{dS} = ?
$$

You may use one of the following approaches:

#### First approach
Using ex.2 and the fact:
$$
|| X ||_F^2 = tr(XX^T)
$$
it is easy to derive gradients (you can find it in one of the refs).

#### Second approach
You can use *slightly different techniques* if they suits you. Take a look at this derivation:
<img src="https://github.com/girafe-ai/ml-course/blob/23f_basic/homeworks/lab01_ml_pipeline/grad.png?raw=1">
(excerpt from [Handbook of blind source separation, Jutten, page 517](https://books.google.ru/books?id=PTbj03bYH6kC&printsec=frontcover&dq=Handbook+of+Blind+Source+Separation&hl=en&sa=X&ved=0ahUKEwi-q_apiJDLAhULvXIKHVXJDWcQ6AEIHDAA#v=onepage&q=Handbook%20of%20Blind%20Source%20Separation&f=false), open for better picture).

#### Third approach
And finally we can use chain rule!
let $ F = AS $

**Find**
$$
\frac{dJ}{dF} =  
$$
and
$$
\frac{dF}{dS} =  
$$
(the shape should be $ NM \times RM )$.

Now it is easy do get desired gradients:

We can write
$$
J = \|X - AS\|_F^2 = \mathrm{tr}\!\big((X - AS)^\top (X - AS)\big).
$$

Expanding and keeping only terms that depend on $S$:
$$
J = \mathrm{tr}(S^\top A^\top A S) - 2\,\mathrm{tr}(S^\top A^\top X) + \text{const}.
$$

Using matrix derivative rules
$$
\frac{\partial}{\partial S}\,\mathrm{tr}(S^\top B S) = (B + B^\top)S, \quad
\frac{\partial}{\partial S}\,\mathrm{tr}(S^\top C) = C,
$$
and the fact that $A^\top A$ is symmetric, we get
$$
\frac{\partial}{\partial S}\,\mathrm{tr}(S^\top A^\top A S) = 2A^\top A S,
$$
$$
\frac{\partial}{\partial S}\Big(-2\,\mathrm{tr}(S^\top A^\top X)\Big) = -2A^\top X.
$$

Therefore
$$
{\frac{dJ}{dS} = 2A^\top(AS - X).}
$$


In [ ]:
def J(X, A, S):
    R = X - A @ S
    return np.sum(R * R)


def dJ_dS(X, A, S):
    return 2 * A.T @ (A @ S - X)


def numeric_grad_S(X, A, S, eps=1e-6):
    G = np.zeros_like(S)
    for i in range(S.shape[0]):
        for j in range(S.shape[1]):
            S1 = S.copy()
            S2 = S.copy()
            S1[i, j] += eps
            S2[i, j] -= eps
            G[i, j] = (J(X, A, S1) - J(X, A, S2)) / (2 * eps)
    return G


np.random.seed(0)
N, R, M = 5, 3, 4
X = np.random.randn(N, M)
A = np.random.randn(N, R)
S = np.random.randn(R, M)

g_analytic = dJ_dS(X, A, S)
g_numeric = numeric_grad_S(X, A, S)

print("allclose:", np.allclose(g_analytic, g_numeric, atol=1e-5))


allclose: True


## 2. kNN questions
Here come the questions from the assignment0_01. Please, refer to the assignment0_01 to get the context of the questions.

### Question 1

Notice the structured patterns in the distance matrix, where some rows or columns are visible brighter. (Note that with the default color scheme black indicates low distances while white indicates high distances.)

- What in the data is the cause behind the distinctly bright rows?
- What causes the columns?

*Your Answer:*

- **Bright rows:**  
  These rows correspond to **test samples that are far from almost all training samples**.  
  In the vehicle dataset this means the object is an **outlier** (unusual shape, brightness, or noise).  
  Because the sample is far from everyone, all distances in its row are large → the row appears bright.

- **Bright columns:**  
  These columns correspond to **training samples that are far from almost all test samples**.  
  Such training samples are also **outliers** in the dataset.  
  Since all distances to them are large, the entire column becomes bright.

### Question 2

We can also use other distance metrics such as L1 distance.
For pixel values $p_{ij}^{(k)}$ at location $(i,j)$ of some image $I_k$,

the mean $\mu$ across all pixels over all images is $$\mu=\frac{1}{nhw}\sum_{k=1}^n\sum_{i=1}^{h}\sum_{j=1}^{w}p_{ij}^{(k)}$$
And the pixel-wise mean $\mu_{ij}$ across all images is
$$\mu_{ij}=\frac{1}{n}\sum_{k=1}^np_{ij}^{(k)}.$$
The general standard deviation $\sigma$ and pixel-wise standard deviation $\sigma_{ij}$ is defined similarly.

Which of the following preprocessing steps will not change the performance of a Nearest Neighbor classifier that uses L1 distance? Select all that apply.
1. Subtracting the mean $\mu$ ($\tilde{p}_{ij}^{(k)}=p_{ij}^{(k)}-\mu$.)
2. Subtracting the per pixel mean $\mu_{ij}$  ($\tilde{p}_{ij}^{(k)}=p_{ij}^{(k)}-\mu_{ij}$.)
3. Subtracting the mean $\mu$ and dividing by the standard deviation $\sigma$.
4. Subtracting the pixel-wise mean $\mu_{ij}$ and dividing by the pixel-wise standard deviation $\sigma_{ij}$.
5. Rotating the coordinate axes of the data.

*Your Answer:*

1, 2, 3


*Your Explanation:*

Transformations 1–3 do not change the behavior of a k-NN classifier that uses L1 distance, because they preserve all pairwise L1 distances up to a constant factor:

- **(1) Subtracting the global mean μ**  
  This is a global shift of all points by the same vector.  
  L1 distances remain identical:
  $$
  \| (x - c) - (y - c) \|_1 = \|x - y\|_1.
  $$

- **(2) Subtracting the pixel-wise mean μ₍ᵢⱼ₎**  
  This is also a shift by a fixed vector in feature space.  
  It does not change any L1 distance.

- **(3) Subtracting μ and dividing by a scalar σ**  
  All distances scale by the same positive constant:
  $$
  \left\|\frac{x - μ}{σ} - \frac{y - μ}{σ}\right\|_1 
  = \frac{1}{σ} \|x - y\|_1.
  $$
  This does not change the ordering of nearest neighbors.

Transformations 4 and 5 change distances:

- **(4)** Pixel-wise standardization applies different scale factors to different coordinates → effectively a weighted L1 distance.  
  The nearest neighbor may change.

- **(5)** Rotations preserve L2 norms but not L1 norms → L1 distances generally change.


In [ ]:
import numpy as np
from sklearn.metrics import pairwise_distances

np.random.seed(0)
X = np.random.randn(20, 5)
x_query = X[0:1]

def neighbors_order(X, x_query):
    d = pairwise_distances(x_query, X, metric='manhattan').ravel()
    return np.argsort(d)

base_order = neighbors_order(X, x_query)

results = {}

mu = X.mean()
X1 = X - mu
results['1'] = np.array_equal(neighbors_order(X1, x_query - mu), base_order)

mu_vec = X.mean(axis=0)
X2 = X - mu_vec
results['2'] = np.array_equal(neighbors_order(X2, x_query - mu_vec), base_order)

sigma = X.std() + 1e-9
X3 = (X - mu) / sigma
results['3'] = np.array_equal(neighbors_order(X3, (x_query - mu) / sigma), base_order)

sigma_vec = X.std(axis=0) + 1e-9
X4 = (X - mu_vec) / sigma_vec
results['4'] = np.array_equal(neighbors_order(X4, (x_query - mu_vec) / sigma_vec), base_order)

Q, _ = np.linalg.qr(np.random.randn(5, 5))
X5 = X @ Q
x5 = x_query @ Q
results['5'] = np.array_equal(neighbors_order(X5, x5), base_order)

results


{'1': True, '2': True, '3': True, '4': False, '5': False}

## Question 3

Which of the following statements about $k$-Nearest Neighbor ($k$-NN) are true in a classification setting, and for all $k$? Select all that apply.
1. The decision boundary (hyperplane between classes in feature space) of the k-NN classifier is linear.
2. The training error of a 1-NN will always be lower than that of 5-NN.
3. The test error of a 1-NN will always be lower than that of a 5-NN.
4. The time needed to classify a test example with the k-NN classifier grows with the size of the training set.
5. None of the above.

*Your Answer:*

2, 4

*Your Explanation:*

1. **False** — k-NN produces nonlinear decision boundaries.
2. **True** — 1-NN has zero training error, so it is always ≤ the training error of 5-NN.
3. **False** — 1-NN often overfits; test error can be higher than 5-NN.
4. **True** — computing distances to all training samples makes prediction time grow with dataset size.
5. **False** — because statements 2 and 4 are correct.
